In [ ]:
library(tidyverse)
library(zellkonverter)

theme_custom <- function(base_size = 12){ 
  theme_bw(base_size = base_size) %+replace%    #replace elements we want to change
    theme(
      panel.grid = element_blank(), 
      strip.background = element_rect(fill = "#F2F2F2", colour = NA)
    )
}
theme_set(theme_custom())
options(repr.plot.width = 8, repr.plot.height = 5, repr.plot.res = 250)

In [ ]:
sce = readH5AD("../data/adata/timecourse.h5ad")

In [ ]:
suppressPackageStartupMessages({
    library(scater)
    library(scran)
    library(scuttle)
    library(SingleCellExperiment)
})

Setting up the object for plotting TGFb trends

In [ ]:
sce$day <- stringr::str_extract(sce$batch, "\\d+")
sce$day <- factor(sce$day, levels = c("6", "8", "30", "90"))

In [ ]:
create_df <- function(marker, sce) {
    df <- data.frame(
        "Day" = retrieveCellInfo(sce, exprs_values = "X", by = "day")$value,
        "Subtype" = retrieveCellInfo(sce, exprs_values = "X", by = "Type")$value,
        "Crypt_Villi_Axis" = retrieveCellInfo(sce, exprs_values = "X", by = "crypt_villi_axis")$value,
        "epithelial_distance_clipped" = retrieveCellInfo(sce, exprs_values = "X", by = "epithelial_distance_clipped")$value,
        "predicted_longitudinal" = retrieveCellInfo(sce, exprs_values = "X", by = "predicted_longitudinal")$value
    )

    for (m in marker) {
        df[m] <- retrieveCellInfo(sce, exprs_values = "X", by = m)$value
    }

    df <- df |> pivot_longer(c("Crypt_Villi_Axis", "epithelial_distance_clipped", "predicted_longitudinal"), names_to = "Axis", values_to = "Axis_value")

    df <- df |> pivot_longer(-c(Axis, Axis_value, Day, Subtype), names_to = "Gene", values_to = "Expression")

    return(df)
}

df <- create_df(c("Tgfb1", "Tgfb2", "Tgfb3"), sce)

head(df)

Plotting trends

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 3, repr.plot.res = 250)
ggplot(filter(df, Axis == "Crypt_Villi_Axis"),
        aes(
          x=Axis_value,
          y=Expression,
          color=Day)
        )+
    geom_smooth(se = FALSE) +
    scale_x_continuous(
      breaks = c(0,1),
      labels = c("bottom", "top"),
      limits = c(0,1)
    ) +
    xlab("Crypt-villus axis") +
    ylab("scaled expression") +
    scale_color_manual(
      values = c("#E15759", "#F28E2B", "#59A14F", "#4E79A7")
    ) +
    facet_wrap( ~ Gene, scales = "free_y", ncol = 3) +
    ggtitle("")